# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [159]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [160]:
df.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [161]:
# import pandas_profiling
# pandas_profiling.ProfileReport(df, minimal=True).to_file("reprot_min.html")

In [162]:
# from statistics import mode
# def mode_value(x):
#     X=x.mode()[0]
#     return X

In [163]:
import statistics 
a = [23, 34, 34, 34, 22]
statistics.mode(a)

34

In [164]:
def mode_v(x):
    X = x.statistics.mode()
    return X

In [165]:
from scipy import stats
def mode_scipy(a):
    X = stats.mode(a)[0]
    return X

In [166]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
#數值型選Age, 類別型選Pclass
df_age = df.pivot_table(index="Pclass", values="Age", aggfunc=["count", "mean", "median","max","min","std", mode_scipy])
df_age

,count,mean,median,max,min,std,mode_scipy
,Age,Age,Age,Age,Age,Age,Age
Pclass,,,,,,,
1,186,38.233441,37.0,80.0,0.92,14.802856,35.0
2,173,29.877630,29.0,70.0,0.67,14.001077,24.0
3,355,25.140620,24.0,74.0,0.42,12.495398,22.0


In [167]:
df = pd.merge(df, df_age, how="left", on="Pclass")
df

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,"(count, Age)","(mean, Age)","(median, Age)","(max, Age)","(min, Age)","(std, Age)","(mode_scipy, Age)"
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,355,25.140620,24.0,74.0,0.42,12.495398,22.0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,186,38.233441,37.0,80.0,0.92,14.802856,35.0
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,355,25.140620,24.0,74.0,0.42,12.495398,22.0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,186,38.233441,37.0,80.0,0.92,14.802856,35.0
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,355,25.140620,24.0,74.0,0.42,12.495398,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,173,29.877630,29.0,70.0,0.67,14.001077,24.0
887,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,186,38.233441,37.0,80.0,0.92,14.802856,35.0
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,355,25.140620,24.0,74.0,0.42,12.495398,22.0
889,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,186,38.233441,37.0,80.0,0.92,14.802856,35.0


In [168]:
df = df.select_dtypes(["int", "float"])
df
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
# num_features = []
# for dtype, feature in zip(df.dtypes, df.columns):
#     if dtype == 'float64' or dtype == 'int64':
#         num_features.append(feature)
# print(f'{len(num_features)} Numeric Features : {num_features}\n')

# # 削減文字型欄位, 只剩數值型欄位
# df = df[num_features]
df = df.fillna(-1)

#df.head()

# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [181]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_ori = df.drop(df.columns[6:13], axis=1)
MMEncoder = MinMaxScaler()
df_scaler = MinMaxScaler().fit_transform(df_ori)
train_X = df_scaler
estimator = LogisticRegression()
Score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
Score

0.6903144812001758

In [182]:
df_ori

,Pclass,Age,SibSp,Parch,Fare,"(count, Age)"
0,3,22.0,1,0,7.2500,355
1,1,38.0,1,0,71.2833,186
2,3,26.0,0,0,7.9250,355
3,1,35.0,1,0,53.1000,186
4,3,35.0,0,0,8.0500,355
...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,173
887,1,19.0,0,0,30.0000,186
888,3,-1.0,1,2,23.4500,355
889,1,26.0,0,0,30.0000,186


In [170]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
MMEncoder = MinMaxScaler()
df_scaler = MinMaxScaler().fit_transform(df)
train_X = df_scaler
estimator = LogisticRegression()
Score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
Score

0.6948088632226478

In [183]:
df

,Pclass,Age,SibSp,Parch,Fare,"(count, Age)","(mean, Age)","(median, Age)","(max, Age)","(min, Age)","(std, Age)","(mode_scipy, Age)"
0,3,22.0,1,0,7.2500,355,25.140620,24.0,74.0,0.42,12.495398,22.0
1,1,38.0,1,0,71.2833,186,38.233441,37.0,80.0,0.92,14.802856,35.0
2,3,26.0,0,0,7.9250,355,25.140620,24.0,74.0,0.42,12.495398,22.0
3,1,35.0,1,0,53.1000,186,38.233441,37.0,80.0,0.92,14.802856,35.0
4,3,35.0,0,0,8.0500,355,25.140620,24.0,74.0,0.42,12.495398,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,173,29.877630,29.0,70.0,0.67,14.001077,24.0
887,1,19.0,0,0,30.0000,186,38.233441,37.0,80.0,0.92,14.802856,35.0
888,3,-1.0,1,2,23.4500,355,25.140620,24.0,74.0,0.42,12.495398,22.0
889,1,26.0,0,0,30.0000,186,38.233441,37.0,80.0,0.92,14.802856,35.0
